<a href="https://colab.research.google.com/github/profteachkids/CHE4071_Fall2023/blob/main/TransferFunctionAlgebra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
from plotly.subplots import make_subplots
import scipy as sp
from numpy.polynomial.polynomial import Polynomial as P
from scipy.signal import TransferFunction as TF
from scipy.signal import ZerosPolesGain as ZPK

In [360]:
def polynd2TF(n,d):
     return TF(n.coef[::-1], d.coef[::-1])

def tf2poly(tf):
    if type(tf) in (int,float):
        return tf, 1.
    return P(tf.num[::-1]), P(tf.den[::-1])

def tf_mult(tf1,tf2):
    n1, d1 = tf2poly(tf1)
    n2, d2 = tf2poly(tf2)
    return polynd2TF(n1*n2, d1*d2)

def tf_div(tf1,tf2):
    n1, d1 = tf2poly(tf1)
    n2, d2 = tf2poly(tf2)

    return polynd2TF(n1*d2, d1*n2)

def tf_add(tf1, tf2):
    n1, d1 = tf2poly(tf1)
    n2, d2 = tf2poly(tf2)

    return polynd2TF(n1*d2 + n2*d1, d1*d2)

def tf_feedback(tf_f, tf_b):
    nf, df = tf2poly(tf_f)
    nb, db = tf2poly(tf_b)
    return polynd2TF(nf*db, df*db + nf*nb)

def pid(Kc, taui, taud):
    return TF([Kc*taud*taui, Kc*taui, Kc], [taui, 0])

In [361]:
def delay_pade(theta, n):
    f = 1
    c = [1]
    for i in range(1,n):
        f*=-theta/i
        c.append(f)
    p,q=sp.interpolate.pade(c, n//2)

    return TF(p.coef,q.coef)

In [362]:
tf_process = tf_mult(delay_pade(1,4),tf_mult(TF([1.],[2.,1.]), TF([1.],[2.,1.])))
tf_process

TransferFunctionContinuous(
array([-0.5,  1.5]),
array([ 1.  ,  5.  , 10.25,  7.  ,  1.5 ]),
dt: None
)

In [363]:
Kc=1
taui=5
taud=0.
tf_forward = tf_mult(pid(Kc, taui, taud), tf_process)
tf_closed_loop = tf_feedback(tf_forward, 1)

/usr/local/lib/python3.10/dist-packages/scipy/signal/_filter_design.py:1746: BadCoefficients:

Badly conditioned filter coefficients (numerator): the results may be meaningless



In [364]:
t=np.linspace(0,50,500)
x,y=tf_closed_loop.step(T=t)
fig=make_subplots()
fig.add_scatter(x=x,y=y.real)
fig.update_layout(width=800,height=600,template='plotly_dark')